# NYC_cabs

In [6]:
#Importation des Modules nécessaires
import pandas as pd


#Pour que python reconnaisse un dossier comme un module il faut ajouter en son sein un fichier __init__.py 
#et également l'ajouter au PATH

import sys
sys.path.insert(0, "/home/apprenant/PycharmProjects/NYC_cabs")


#On peut ensuite importer le dossier comme un module

from src.utils import distanceGPS,deg2rad


## 1. Exploration des données

## 1.1 calculer la vitesse moyenne de chaque trajet (en km/h) 

In [11]:
# pour ce calcul il faut la distance en km, le temps de trajet en heure.


# cooordonnées GPS en radians du 1er point (ici, mairie de Tours)
latA = deg2rad(47.390668)  # Nord
longA = deg2rad(0.689319)  # Est

# cooordonnées GPS en radians du 2ème point (ici, mairie de Limoges)
latB = deg2rad(45.826516)  # Nord
longB = deg2rad(1.260290)  # Est

dist_km = (distanceGPS(latA, longA, latB, longB))/1000
print(int(dist_km),"km")

179 km


## 1.2 calculer le nombre de trajets effectués en fonction du jour de la semaine 

## 1.3 calculer le nombre de trajets effectués en fonction de l’horaire de la journée par tranche de 4h.

## 1.4 calculer le nombre de km parcourus par jour de la semaine

## 2. Choix du modèle de machine learning

## 3. Préparation des données

### 3.a échantillonage

### 3.b Sélection de la variable cible et des variables à explicatives

## 4. Division du jeu de données en training/validation/testing sets

## 5. Apprentissage sur le training set

#### On implémente la regression linéaire

#### On obtient ainsi les parmètres β1 et β0 de notre modèle

#### On peut à présent réaliser des prédictions

#### enfin on peut représenter notre modèle linéaire

## 6. Evaluation du modèle sur le testing set

#### évaluation à partir du R2

#### évaluation à partir du RMSE

#### évaluation graphique